In [7]:
# run_one_more_testcase.py
import os
import sys
import pickle
import logging
from itertools import groupby
from typing import List, Dict, Optional

import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

# --- IMPORTANT: same aux_functions imports as your training script ---
sys.path.append("../aux_functions")  # adjust if needed
from functions_datasets import CARAVAN as DataBase
from functions_datasets import validate_samples
from functions_evaluation import nse
from functions_aux import create_folder


In [8]:
# --------------------------
# CONFIG (edit these)
# --------------------------

# Folder of an EXISTING run (seed folder) that already contains scaler.pickle and checkpoint.pth
RUN_FOLDER = "../results/with_discharge_haicore/seed_77584"  # <-- change to your seed folder

# If you prefer loading a specific epoch file (your script saved it as RUN_FOLDER/epoch_X), set this.
# Otherwise the script will load RUN_FOLDER/checkpoint.pth.
EPOCH_STATE_PATH = ""  # e.g. "../results/.../seed_77584/epoch_5"  (leave "" to use checkpoint.pth)

# Data
PATH_DATA = "../../datasets"  # same as your script
TESTING_PERIOD = ["1993-01-01", "1995-12-31"]

# One more test case: basin ids you want to test now
BASIN_IDS_TO_TEST = ["neck_1"]  # put your basin id(s) here

# Same variable lists as your script
DYNAMIC_INPUT = [
    "temperature_2m_mean",
    "surface_net_solar_radiation_mean",
    "surface_net_thermal_radiation_mean",
    "qobs_lead",
    "total_precipitation_sum",
]
STATIC_INPUT = ["area", "ele_mt_sav", "frac_snow", "pet_mm_syr"]
TARGET = ["eobs_new"]

MODEL_HYPER_PARAMETERS = {
    "input_size": len(DYNAMIC_INPUT) + len(STATIC_INPUT),
    "no_of_layers": 1,
    "seq_length": 365,
    "hidden_size": 64,
    "batch_size": 256,  # not critical here (we use batch_sampler per basin)
    "drop_out": 0.4,
    "set_forget_gate": 3,
}

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Output
OUT_DIR = os.path.join(RUN_FOLDER, "one_more_testcase")
create_folder(OUT_DIR)

# Logging
log_file = os.path.join(OUT_DIR, "one_more_testcase.log")
logging.basicConfig(
    filename=log_file,
    filemode="w",
    level=logging.INFO,
    format="%(asctime)s %(levelname)s: %(message)s",
)
logging.info("Starting one-more-testcase script")

Folder '../results/with_discharge_haicore/seed_77584/one_more_testcase' created successfully.


In [3]:

# --------------------------
# Dataset + Model (copied/minimized from your file)
# --------------------------

class BaseDataset(Dataset):
    def __init__(
        self,
        dynamic_input: List[str],
        static_input: List[str],
        target: List[str],
        sequence_length: int,
        time_period: List[str],
        path_entities: str,
        path_data: str,
        path_additional_features: str = "",
        forcings: List[str] = None,
        check_NaN: bool = True,
    ):
        self.time_period = time_period
        self.dynamic_input = dynamic_input
        self.target = target
        self.sequence_length = sequence_length

        entities_ids = np.loadtxt(path_entities, dtype="str").tolist()
        self.entities_ids = [entities_ids] if isinstance(entities_ids, str) else entities_ids

        self.sequence_data = {}
        self.df_ts = {}
        self.scaler = {}
        self.basin_std = {}
        self.valid_entities = []

        self.static_input = static_input
        if static_input:
            self.df_attributes = self._load_attributes(path_data)

        if path_additional_features:
            self.additional_features = self._load_additional_features(path_additional_features)

        for basin_id in self.entities_ids:
            df_ts = self._load_data(path_data=path_data, catch_id=basin_id)

            if path_additional_features:
                df_ts = pd.concat([df_ts, self.additional_features[basin_id]], axis=1)

            start_date = pd.to_datetime(self.time_period[0], format="%Y-%m-%d")
            end_date = pd.to_datetime(self.time_period[1], format="%Y-%m-%d")
            freq = pd.infer_freq(df_ts.index)
            warmup_start_date = start_date - (self.sequence_length - 1) * pd.tseries.frequencies.to_offset(freq)

            df_ts = df_ts.loc[warmup_start_date:end_date, self.dynamic_input + self.target]

            full_range = pd.date_range(start=warmup_start_date, end=end_date, freq=freq)
            df_ts = df_ts.reindex(full_range)

            flag = validate_samples(
                x=df_ts.loc[:, self.dynamic_input].values,
                y=df_ts.loc[:, self.target].values,
                attributes=self.df_attributes.loc[basin_id].values if static_input else None,
                seq_length=self.sequence_length,
                check_NaN=check_NaN,
            )

            valid_samples = np.argwhere(flag == 1)
            self.valid_entities.extend([(basin_id, int(f[0])) for f in valid_samples])

            if valid_samples.size > 0:
                self.df_ts[basin_id] = df_ts
                self.sequence_data[basin_id] = {}
                self.sequence_data[basin_id]["x_d"] = torch.tensor(
                    df_ts.loc[:, self.dynamic_input].values, dtype=torch.float32
                )
                self.sequence_data[basin_id]["y"] = torch.tensor(
                    df_ts.loc[:, self.target].values, dtype=torch.float32
                )
                if self.static_input:
                    self.sequence_data[basin_id]["x_s"] = torch.tensor(
                        self.df_attributes.loc[basin_id].values, dtype=torch.float32
                    )

    def __len__(self):
        return len(self.valid_entities)

    def __getitem__(self, idx):
        basin, i = self.valid_entities[idx]
        x_lstm = self.sequence_data[basin]["x_d"][i - self.sequence_length + 1 : i + 1, :]
        if self.static_input:
            x_s = self.sequence_data[basin]["x_s"].repeat(x_lstm.shape[0], 1)
            x_lstm = torch.cat([x_lstm, x_s], dim=1)
        y_obs = self.sequence_data[basin]["y"][i]
        return x_lstm, y_obs

    def _load_attributes(self, path_data: str) -> pd.DataFrame:
        df_attributes = DataBase.read_attributes(path_data=path_data)
        df_attributes = df_attributes.loc[self.entities_ids, self.static_input]
        return df_attributes

    def _load_data(self, path_data: str, catch_id: str) -> pd.DataFrame:
        return DataBase.read_data(path_data=path_data, catch_id=catch_id)

    def _load_additional_features(self, path_additional_features: str) -> Dict[str, pd.DataFrame]:
        with open(path_additional_features, "rb") as f:
            return pickle.load(f)

    def standardize_data(self, standardize_output: bool = True):
        for basin in self.sequence_data.values():
            basin["x_d"] = (basin["x_d"] - self.scaler["x_d_mean"]) / self.scaler["x_d_std"]
            if self.static_input:
                basin["x_s"] = (basin["x_s"] - self.scaler["x_s_mean"]) / self.scaler["x_s_std"]
            if standardize_output:
                basin["y"] = (basin["y"] - self.scaler["y_mean"]) / self.scaler["y_std"]


class Cuda_LSTM(nn.Module):
    def __init__(self, model_hyper_parameters):
        super().__init__()
        self.hidden_units = model_hyper_parameters["hidden_size"]
        self.num_layers = model_hyper_parameters["no_of_layers"]

        self.lstm = nn.LSTM(
            input_size=model_hyper_parameters["input_size"],
            hidden_size=model_hyper_parameters["hidden_size"],
            batch_first=True,
            num_layers=model_hyper_parameters["no_of_layers"],
        )
        self.dropout = nn.Dropout(model_hyper_parameters["drop_out"])
        self.linear = nn.Linear(model_hyper_parameters["hidden_size"], 1)

    def forward(self, x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units, device=x.device, dtype=torch.float32)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units, device=x.device, dtype=torch.float32)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]
        out = self.dropout(out)
        return self.linear(out)


def _strip_module_prefix(state_dict: Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]:
    # Helps loading whether weights were saved from DataParallel or not
    if not any(k.startswith("module.") for k in state_dict.keys()):
        return state_dict
    return {k.replace("module.", "", 1): v for k, v in state_dict.items()}


def load_model(run_folder: str) -> nn.Module:
    model = Cuda_LSTM(MODEL_HYPER_PARAMETERS).to(DEVICE)

    # Apply same forget-gate bias tweak (safe even for inference-only)
    lstm_layer = model.lstm
    hidden_size = MODEL_HYPER_PARAMETERS["hidden_size"]
    lstm_layer.bias_hh_l0.data[hidden_size : 2 * hidden_size] = MODEL_HYPER_PARAMETERS["set_forget_gate"]

    # Load weights
    if EPOCH_STATE_PATH:
        raw = torch.load(EPOCH_STATE_PATH, map_location=DEVICE)
        # epoch_X was saved as state_dict in your script
        try:
            model.load_state_dict(_strip_module_prefix(raw), strict=True)
        except RuntimeError:
            # maybe it includes module.* keys already suitable for DataParallel
            dp = nn.DataParallel(model)
            dp.load_state_dict(raw, strict=True)
            model = dp.module
    else:
        ckpt_path = os.path.join(run_folder, "checkpoint.pth")
        ckpt = torch.load(ckpt_path, map_location=DEVICE)
        sd = ckpt["model_state_dict"]
        try:
            model.load_state_dict(_strip_module_prefix(sd), strict=True)
        except RuntimeError:
            dp = nn.DataParallel(model)
            dp.load_state_dict(sd, strict=True)
            model = dp.module

    model.eval()
    return model


def write_entities_file(out_dir: str, basin_ids: List[str]) -> str:
    path = os.path.join(out_dir, "entities_one_more_test.txt")
    with open(path, "w", encoding="utf-8") as f:
        for bid in basin_ids:
            f.write(f"{bid}\n")
    return path


In [9]:
def main():
    # 1) Load scaler
    scaler_path = os.path.join(RUN_FOLDER, "scaler.pickle")
    with open(scaler_path, "rb") as f:
        scaler = pickle.load(f)
    logging.info("Loaded scaler.pickle")

    # 2) Build dataset for your new test basin(s)
    path_entities = write_entities_file(OUT_DIR, BASIN_IDS_TO_TEST)
    test_dataset = BaseDataset(
        dynamic_input=DYNAMIC_INPUT,
        static_input=STATIC_INPUT,
        target=TARGET,
        sequence_length=MODEL_HYPER_PARAMETERS["seq_length"],
        time_period=TESTING_PERIOD,
        path_entities=path_entities,
        path_data=PATH_DATA,
        check_NaN=False,
    )
    test_dataset.scaler = scaler
    test_dataset.standardize_data(standardize_output=False)  # IMPORTANT: same as your testing part

    # Basin-wise batches (same logic as your script)
    test_batches = [
        [index for index, _ in group]
        for _, group in groupby(enumerate(test_dataset.valid_entities), lambda x: x[1][0])
    ]
    test_loader = DataLoader(dataset=test_dataset, batch_sampler=test_batches)
    logging.info(f"Batches in testing: {len(test_loader)}")

    valid_basins = [next(group)[0] for key, group in groupby(test_dataset.valid_entities, key=lambda x: x[0])]
    valid_entity_per_basin = [
        [i for _, i in group] for key, group in groupby(test_dataset.valid_entities, key=lambda x: x[0])
    ]

    # 3) Load trained model
    model = load_model(RUN_FOLDER)
    logging.info("Loaded trained model weights")

    # 4) Predict
    test_results = {}
    with torch.no_grad():
        for i, (x_lstm, y) in enumerate(test_loader):
            x_lstm = x_lstm.to(DEVICE)
            y_sim = model(x_lstm)

            # de-standardize model output (exactly like your script)
            y_sim = y_sim * test_dataset.scaler["y_std"].to(DEVICE) + test_dataset.scaler["y_mean"].to(DEVICE)

            basin_id = valid_basins[i]
            df_ts = test_dataset.df_ts[basin_id].iloc[valid_entity_per_basin[i]]
            df_new = pd.DataFrame(
                data={
                    "y_obs": y.flatten().cpu().numpy(),
                    "y_sim": y_sim.flatten().cpu().numpy(),
                },
                index=df_ts.index,
            )
            test_results[basin_id] = df_new

    # 5) Save outputs + NSE
    out_pred = os.path.join(OUT_DIR, "predictions")
    create_folder(out_pred)

    # per-basin CSVs
    for bid, df in test_results.items():
        df.to_csv(os.path.join(out_pred, f"{bid}_y_obs_y_sim.csv"), index=True)

    # NSE (per basin)
    nse_vals = nse(df_results=test_results, average=False)
    df_nse = pd.DataFrame({"basin_id": valid_basins, "NSE": np.round(nse_vals, 3)}).set_index("basin_id")
    df_nse.to_csv(os.path.join(OUT_DIR, "NSE_one_more_testcase.csv"))

    # also write combined wide format like your script
    y_sim = pd.concat([pd.DataFrame({k: v["y_sim"]}) for k, v in test_results.items()], axis=1)
    y_obs = pd.concat([pd.DataFrame({k: v["y_obs"]}) for k, v in test_results.items()], axis=1)
    y_sim.to_csv(os.path.join(OUT_DIR, "y_sim_one_more_testcase.csv"), index=True)
    y_obs.to_csv(os.path.join(OUT_DIR, "y_obs_one_more_testcase.csv"), index=True)

    print("Done.")
    print(f"Saved results in: {OUT_DIR}")
    print(df_nse)


if __name__ == "__main__":
    main()


Folder '../results/with_discharge_haicore/seed_77584/one_more_testcase/predictions' created successfully.
Done.
Saved results in: ../results/with_discharge_haicore/seed_77584/one_more_testcase
            NSE
basin_id       
neck_1    0.614


In [10]:
from pathlib import Path
import pandas as pd

# -----------------------
# EDIT THESE
# -----------------------
ROOT = Path("../results/with_discharge_haicore")   # parent folder that contains seed_*/ dirs
SEED_DIRS = ["seed_47998", "seed_6697", "seed_77584"]   # your 3 seeds
BASIN_FILE = "neck_1_y_obs_y_sim.csv"              # file name inside predictions/
REL_PATH = Path("one_more_testcase/predictions")   # relative path inside each seed dir

OUT_DIR = ROOT / "ensemble_3seeds" / "one_more_testcase" / "predictions"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# -----------------------
# Load per-seed predictions
# -----------------------
dfs = []
for sd in SEED_DIRS:
    f = ROOT / sd / REL_PATH / BASIN_FILE
    if not f.exists():
        raise FileNotFoundError(f"Missing: {f}")
    df = pd.read_csv(f, index_col=0, parse_dates=True).sort_index()
    if "y_sim" not in df.columns:
        raise ValueError(f"{f} has no 'y_sim' column. Columns: {list(df.columns)}")
    dfs.append(df[["y_sim"]].rename(columns={"y_sim": sd}))

# -----------------------
# Align on common timestamps
# -----------------------
common_index = dfs[0].index
for df in dfs[1:]:
    common_index = common_index.intersection(df.index)

dfs = [df.loc[common_index] for df in dfs]

# -----------------------
# Ensemble stats
# -----------------------
y_sim_all = pd.concat(dfs, axis=1)  # columns = seed names
y_sim_mean = y_sim_all.mean(axis=1).to_frame("y_sim_mean")
y_sim_std  = y_sim_all.std(axis=1).to_frame("y_sim_std")
y_sim_min  = y_sim_all.min(axis=1).to_frame("y_sim_min")
y_sim_max  = y_sim_all.max(axis=1).to_frame("y_sim_max")

# optionally also keep y_obs from the first seed (should be identical across seeds)
f0 = ROOT / SEED_DIRS[0] / REL_PATH / BASIN_FILE
y_obs = pd.read_csv(f0, index_col=0, parse_dates=True).sort_index().loc[common_index, ["y_obs"]]

df_out = pd.concat([y_obs, y_sim_mean, y_sim_std, y_sim_min, y_sim_max], axis=1)

# -----------------------
# Save
# -----------------------
out_file = OUT_DIR / BASIN_FILE.replace("_y_obs_y_sim.csv", "_ensemble_y_obs_y_sim.csv")
df_out.to_csv(out_file)

# also save the per-seed matrix (handy for debugging/plots)
y_sim_all.to_csv(OUT_DIR / BASIN_FILE.replace("_y_obs_y_sim.csv", "_seeds_y_sim_matrix.csv"))

print("Saved ensemble file:", out_file.resolve())
print("Saved seed-matrix file:", (OUT_DIR / BASIN_FILE.replace("_y_obs_y_sim.csv", "_seeds_y_sim_matrix.csv")).resolve())
print(df_out.head())



Saved ensemble file: /hkfs/home/haicore/iwu/as2023/lstm_backward/results/with_discharge_haicore/ensemble_3seeds/one_more_testcase/predictions/neck_1_ensemble_y_obs_y_sim.csv
Saved seed-matrix file: /hkfs/home/haicore/iwu/as2023/lstm_backward/results/with_discharge_haicore/ensemble_3seeds/one_more_testcase/predictions/neck_1_seeds_y_sim_matrix.csv
            y_obs  y_sim_mean  y_sim_std  y_sim_min  y_sim_max
1993-01-01    0.0    0.107385   0.101855   0.010355   0.213460
1993-01-02    0.0    0.104652   0.105122   0.002297   0.212338
1993-01-03    0.0    0.191113   0.130427   0.060833   0.321687
1993-01-04    0.0    0.164880   0.186867  -0.048891   0.297178
1993-01-05    0.0    0.242421   0.148036   0.078075   0.365304
